In [0]:
import os
import requests

In [0]:
CATALOG = "marcell"
SCHEMA = "marine_ai_poc"
PROJECT_ID = "EN010109"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.data")

pdf_dir = f"/Volumes/{CATALOG}/{SCHEMA}/data/pdf_files"

In [0]:
dbutils.fs.mkdirs(pdf_dir)

In [0]:
df_reference = spark.table(f"{CATALOG}.{SCHEMA}.document_reference").where(f"project_id = '{PROJECT_ID}'").toPandas()

In [0]:
df_reference["filename"] = None
df_reference["pdf_path"] = None

for i, d in df_reference.iterrows():
    pdf_url = d["pdf_url"]
    filename = pdf_url.split("/")[-1][:16].replace("-", "_") + ".pdf"
    pdf_path = os.path.join(pdf_dir, filename)
    response = requests.get(pdf_url)
    with open(pdf_path, "wb") as pdf_file:
        pdf_file.write(response.content)
    
    df_reference.loc[i, "filename"] = filename
    df_reference.loc[i, "pdf_path"] = pdf_path

In [0]:
spark.createDataFrame(df_reference).write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.document_reference_with_paths")